In [1]:
%matplotlib inline
from sklearn.metrics import mutual_info_score
import numpy as np
import pandas as pd
import random
import itertools
import math
from math import log
import scipy as sp

In [2]:
def contingency_matrix(labels_true, labels_pred):
   

    classes, class_idx = np.unique(labels_true, return_inverse=True)
    clusters, cluster_idx = np.unique(labels_pred, return_inverse=True)
    n_classes = classes.shape[0]
    n_clusters = clusters.shape[0]
    
    contingency = sp.sparse.coo_matrix((np.ones(class_idx.shape[0]),
                                 (class_idx, cluster_idx)),
                                shape=(n_classes, n_clusters),
                                dtype=np.int)

    contingency = contingency.toarray()
 
    return contingency


def joint_entropy(labels_true, labels_pred):

    contingency = contingency_matrix(labels_true, labels_pred)
    nzx, nzy = np.nonzero(contingency)
    nz_val = contingency[nzx, nzy]
    contingency_sum = contingency.sum()

    contingency_sum = contingency.sum()
    log_contingency_nm = np.log(nz_val)
    nz_val = map(float,nz_val)
    contingency_nm = nz_val / contingency_sum
    mi = - contingency_nm * (log_contingency_nm - log(contingency_sum))    
    return mi.sum()

def MI_distance(labels_true, labels_pred):
    MI = mutual_info_score(labels_true, labels_pred)
    H_xy = joint_entropy(labels_true, labels_pred)
    return (H_xy- MI)/H_xy

In [3]:
def Largest_dis_based_MI(seed,stocks,data):
    Largest_dis = - np.inf
    Largest_dis_stock = ''
    for s in stocks:
        s_iter = data.loc[s]
        s_seed = data.loc[seed]
        dis = MI_distance(s_seed,s_iter)
        if Largest_dis < dis:
            Largest_dis = dis
            Largest_dis_stock = s
    return Largest_dis_stock, Largest_dis

def Creat_bicluster(n):
    Bicluster = []
    Bicluster_scores = []
    for i in range(1,n+1):
        Bicluster.append(['B_'+str(i)])
        Bicluster_scores.append(['B_scores'+str(i)])
    return Bicluster, Bicluster_scores

In [4]:
def Create_pairs(data):
    stocks = list(data.KEY.unique())
    all_comb = list(itertools.combinations(stocks, 2))
    return all_comb

def Compute_All_MI(data):
    Dis_pairs = []
    pairs = Create_pairs(data)
    data_with_index = data.set_index('KEY')
    for e in pairs:
        dis = MI_distance(data_with_index.loc[e[0]],data_with_index.loc[e[1]])
        Dis_pairs.append(dis)
    
    S1 = []
    S2 = []
    for i in range(len(pairs)):
        S1.append(pairs[i][0])
        S2.append(pairs[i][1])
        
    Dis_pairs_df = pd.DataFrame([S1, S2, Dis_pairs])
    Dis_pairs_df = Dis_pairs_df.T
    Dis_pairs_df.columns=['S1', 'S2', 'Score']
    #Dis_pairs_df.to_csv('MI_pairs_score.csv',index=False)
    Sorted = Dis_pairs_df.sort_values(by=['Score'],ascending=False)
    Largest_record = Sorted[:1]
    seed1 = Largest_record.S1
    seed2 = Largest_record.S2
    return seed1.tolist()[0], seed2.tolist()[0]

In [5]:
def Initial_seed(data, n):
    #print n
    Bicluster, Bicluster_scores = Creat_bicluster(n)
    seed0, seed1 = Compute_All_MI(data)
    seed_list = []
    Bicluster[0].append(seed0)
    Bicluster[1].append(seed1)
    seed_list.append(seed0)
    seed_list.append(seed1)
    
    stocks = list(data.KEY.unique())
    data_with_index = data.set_index('KEY')
    stocks_bucket = list(set(stocks)-set(seed_list))
    #print 'initial stockbucket', stocks_bucket
    for i in range(2, n):
       # print i
        largest_dis_stock=''
        largest_dis = - np.inf
        #print 'seed list', seed_list
        
        for stock in stocks_bucket:
            dis_total = 0
            j = 0
            for seed in seed_list:
               # print 'seed', seed
                s_iter = data_with_index.loc[stock]
                s_seed = data_with_index.loc[seed]
                dis = MI_distance(s_seed,s_iter)
                dis_total += dis
                j += 1
            Average_dis = dis_total*1./j
        
           # print 'average_MI_score',stock, Average_MI_score
            if largest_dis < Average_dis:
                largest_dis = Average_dis
                largest_dis_stock = stock
                #print 'lowest_score', lowest_score
                #print 'lowest_score_stock', lowest_score_stock
        #print 'lowest_score_stock', lowest_score_stock
        #print 'lowest_score', lowest_score
        stocks_bucket.remove(largest_dis_stock)
        #print stocks_bucket
        seed_list.append(largest_dis_stock)
        Bicluster[i].append(largest_dis_stock)
    #print stocks_bucket
    return stocks_bucket, Bicluster, Bicluster_scores, n 

def Smallest_dis_MI(data, n):
    
    stocks_bucket, Bicluster, Bicluster_scores, n = Initial_seed(data, n)
    data_with_index = data.set_index('KEY')
    
    for s in stocks_bucket:
        
        s_iter = data_with_index.loc[s]
        
        Smallest_dis_seed = ''
        Smallest_dis = np.inf
        
        for i in range(0,n):
            
            Largest_dis = - np.inf
            stock_in_bicluster = Bicluster[i][1:]
            for s_b in stock_in_bicluster:
                s_in_b = data_with_index.loc[s_b]
                dis = MI_distance(s_in_b,s_iter)
                if Largest_dis < dis:
                    Largest_dis = dis
            bicluster_MI = Largest_dis
            if Smallest_dis > bicluster_MI:
                Smallest_dis = bicluster_MI
                Smallest_dis_seed_index = i
                #print seed
                #rint Highest_MI
        Bicluster[Smallest_dis_seed_index].append(s)
        Bicluster_scores[Smallest_dis_seed_index].append(str(round(Smallest_dis,3)))
    return Bicluster, Bicluster_scores

def Silhouette_score(data, n):
    Bicluster, Bicluster_scores = Smallest_dis_MI(data, n)
    data_with_index = data.set_index('KEY')
    C = 0
    Total_S_i = 0
    for e in Bicluster:
        #print e
        a_i = 0
        B_stocks = e[1:]
        if len(B_stocks) == 1:
            return 0
        for ee in B_stocks:
            #print ee
            ee_data = data_with_index.loc[ee]
            for i in range(len(B_stocks)):
                if B_stocks[i] != ee:
                    temp_data = data_with_index.loc[B_stocks[i]]
                    a_temp = MI_distance(ee_data,temp_data)
                    a_i += a_temp
            
            a_i /= (len(B_stocks)-1)
            #print a_i
           
            
            ## compute b_i
            b_i =  np.inf
            for e_other in Bicluster:
                b_temp = 0
                if e != e_other:
                    B_other_stocks = e_other[1:]
                    for ee_other in B_other_stocks:
                        #print 'ee', ee_other
                        ee_other_data = data_with_index.loc[ee_other]    
                        other_temp = MI_distance(ee_other_data,ee_data)
                        b_temp += other_temp
                    b_temp_average = b_temp / (len(B_other_stocks))
                    #print 'b_aver', b_temp_average
                    if b_i >  b_temp_average:
                        b_i = b_temp_average
            S_i =  (b_i - a_i) / (max(b_i, a_i))
            C += 1
            Total_S_i += S_i
    Average_S_i = Total_S_i / C
    return Average_S_i
     

def determine_N(data):
    Sil_score =[]
    for n in range (2, 32):
        temp = Silhouette_score(data, n)
        Sil_score.append(temp)
    
    Max_value_index = Sil_score.index(max(Sil_score))
    
    # optimal number of cluster
    n = Max_value_index + 2
    
    return n

In [6]:
def get_cluster(data):
    
    n = determine_N(data)
    print ("optimal n: ",n)
    Bicluster, Bicluster_scores = Smallest_dis_MI(data, n)
    
    return pd.DataFrame(Bicluster), len(Bicluster)

In [16]:
#datelist = pd.read_csv('./Quantile_10percent/stockdatelist.csv')
#date = datelist.date.tolist()
#date1 = date[120:160]   
#N_list = []
#for e in date1:
#    print (e)
    #infile = '/Volumes/Haitao_SSD/2013sp100ticker_by_ticker/8Mutual_info/Quantile_10percent/stockreturn1min'+str(e)+'_digitized.csv'
    #outfilepath = '/Volumes/Haitao_SSD/2013sp100ticker_by_ticker/8Mutual_info/Quantile_10percent_clusters_sil_dis/'
infile = './stocktest.csv'
outfilepath = './'

data = pd.read_csv(infile)
cluster, number_of_element = get_cluster(data)
cluster.to_csv(outfilepath+'clusters.csv',index=False)

N_list.append(number_of_element)
    
stats= pd.DataFrame([N_list])
stats = stats.transpose()
stats.columns = ['date', 'N']
stats.to_csv(outfilepath+'stats.csv')

<ipython-input-2-1edb00532d80>:12: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  dtype=np.int)


TypeError: unsupported operand type(s) for /: 'map' and 'int'

In [15]:
data

,0,1,2,3,4,5,6,7,8,9,...,84,85,86,87,88,89,90,91,92,93
KEY,AAPL,ABBV,ABT,ACN,AEP,ALL,AMGN,AMZN,APA,APC,...,UPS,USB,UTX,V,VZ,WFC,WMT,XOM,GOOG,WAG
20130102 09:31,1,10,3,10,10,1,2,1,9,1,...,10,3,9,10,1,5,9,1,10,10
20130102 09:32,1,1,10,10,9,1,10,4,9,10,...,10,2,3,1,1,7,1,1,1,10
20130102 09:33,10,5,2,10,7,1,5,10,2,1,...,2,8,3,1,1,2,1,1,4,10
20130102 09:34,10,9,1,4,10,1,2,3,7,2,...,3,8,7,10,2,7,6,10,10,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20130102 15:55,7,1,10,9,10,10,3,5,9,10,...,9,8,2,10,5,7,9,9,4,8
20130102 15:56,4,3,6,1,1,2,1,1,2,1,...,1,3,9,2,1,1,1,1,1,2
20130102 15:57,3,10,9,9,8,4,7,4,9,2,...,3,7,9,7,6,5,5,8,8,3
20130102 15:58,7,10,3,10,10,10,9,9,9,8,...,10,3,8,10,10,10,10,10,7,8
